# Preprocess ModelNet10 data to numpy files

In [4]:
!pip install trimesh pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.5 MB 3.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import trimesh
from tqdm import tqdm

In [2]:
SOURCE_DIR = 'ModelNet10'
OUTPUT_DIR = 'mn10'
CUBE_SIZE = 32

In [3]:
off_list = glob(os.path.join(SOURCE_DIR, '**/*.off'), recursive=True)

In [4]:
def get_split(path):
    return path.split('/')[-2]

def get_name(path):
    return os.path.splitext(os.path.basename(path))[0]

In [5]:
def off_to_voxel(path, matrix_size=64):
    mesh = trimesh.load(path)
    points = mesh.vertices
    size = points.max(axis=0) - points.min(axis=0)
    pitch = size.max() / (matrix_size - 3)
    voxel = mesh.voxelized(pitch, max_iter=50)
    matrix = voxel.matrix.astype(np.uint8)
    orig_shape = matrix.shape
    target_shape = (matrix_size, matrix_size, matrix_size)
    anchor = (np.array(target_shape, dtype=np.int32)
           - np.array(orig_shape, dtype=np.int32)) // 2
    buffer = np.zeros(target_shape, dtype=np.uint8)
    buffer[anchor[0]:anchor[0] + orig_shape[0],
         anchor[1]:anchor[1] + orig_shape[1], 
         anchor[2]:anchor[2] + orig_shape[2]] = matrix
    return buffer

In [6]:
for path in tqdm(off_list):
    split = get_split(path)
    name = get_name(path)
    output_dir = os.path.join(OUTPUT_DIR, str(CUBE_SIZE), split)
    os.makedirs(output_dir, exist_ok=True)
    matrix = off_to_voxel(path, matrix_size=CUBE_SIZE)
    output_path = os.path.join(output_dir, name + '.npy')
    np.save(output_path, matrix)

100%|██████████| 4899/4899 [38:08<00:00,  2.14it/s]  
